Here I will construct a CNN within tensorflow to identify the type of flower, won't actually be running this code here though becuase it requires a GPU, so that'll happen on a remote server but let's get the basics down here!


https://www.kaggle.com/alxmamaev/flowers-recognition

Should now only read in jpegs

In [ ]:
%matplotlib inline


import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import os

Part I
------
Making the dataset

Here I'm making the arrays of each flower type, defining their label, and then making a function to find the max height and width.

Not sure how to find the "nominal" height and width, but the code to find it is in here provided you know what you're looking for (I don't). 

In [ ]:
cd flowers/

In [ ]:
ls

In [ ]:
#Making my daisy set in here

X_daisies = []
y_daisies = []
for file in os.listdir('/Users/noahkasmanoff/MachLearn/Phys476/hw4/flowers/daisy'):
    if file.endswith(".jpg"):
        img = cv2.imread('/Users/noahkasmanoff/MachLearn/Phys476/hw4/flowers/daisy/' + file)
        X_daisies.append(img)
        y_daisies.append([1,0,0,0,0])

        
        
#Now for dandelions
X_dandelions = []
y_dandelions = []
for file in os.listdir('/Users/noahkasmanoff/MachLearn/Phys476/hw4/flowers/dandelion'):
    #print(filename)
    if file.endswith(".jpg"):
        img = cv2.imread('/Users/noahkasmanoff/MachLearn/Phys476/hw4/flowers/dandelion/' + file)
        X_dandelions.append(img)
        y_dandelions.append([0,1,0,0,0])

#Now for roses
X_roses = []
y_roses = []
for file in os.listdir('/Users/noahkasmanoff/MachLearn/Phys476/hw4/flowers/rose'):
    #print(filename)
    if file.endswith(".jpg"):
        img = cv2.imread('/Users/noahkasmanoff/MachLearn/Phys476/hw4/flowers/rose/' + file)
        X_roses.append(img)
        y_roses.append([0,0,1,0,0])

#Now for sunflowers
X_sunflowers = []
y_sunflowers = []
for file in os.listdir('/Users/noahkasmanoff/MachLearn/Phys476/hw4/flowers/sunflower'):
    #print(filename)
    if file.endswith(".jpg"):
        img = cv2.imread('/Users/noahkasmanoff/MachLearn/Phys476/hw4/flowers/sunflower/' + file)
        X_sunflowers.append(img)
        y_sunflowers.append([0,0,0,1,0])

#Last but not least, tulips
X_tulips = []
y_tulips = []
for file in os.listdir('/Users/noahkasmanoff/MachLearn/Phys476/hw4/flowers/tulip'):
    #print(filename)
    if file.endswith(".jpg"):
        img = cv2.imread('/Users/noahkasmanoff/MachLearn/Phys476/hw4/flowers/tulip/' + file)
        X_tulips.append(img)
        y_tulips.append([0,0,0,0,1])

I've now created arrays for each of the flowers, now merging them all together...

In [ ]:
#Merging all at once. 4236 flowers in all. 

X_args = (X_daisies,X_dandelions,X_roses,X_sunflowers,X_tulips)
y_args = (y_daisies,y_dandelions,y_roses,y_sunflowers,y_tulips)
X_flowers = np.concatenate(X_args)
y_flowers = np.concatenate(y_args)



After running the block below we can identify shitty data. Delete here. 
Done!

In [ ]:
maxone = 0
#Don't make this for just daisies, do it for all flowers and then you can make the black space
for i in range(len(X_flowers)):
    flower = X_flowers[i]
    height = np.shape(flower)[1] #*np.shape(daisy)[1]
    #Some er
    #print(i)
    if height>maxone:
        print(i)
        print("hmax  = " +str(height))
        maxone = height


In [ ]:
maxone = 0
#Don't make this for just daisies, do it for all flowers and then you can make the black space
for i in range(len(X_flowers)):
    flower = X_flowers[i]
    width = np.shape(flower)[0] #*np.shape(daisy)[1]
    #Some er
    #print(i)
    if width>maxone:
        print(i)
        print("wmax  = " +str(width))
        maxone = width


In [ ]:
# Resize images
def resize_with_pad(img, img_size):
    height, width, _ = img.shape
    ratio = img_size / max(height, width)
   # print(ratio)
    if ratio < 1:
        img = cv2.resize(img, (int(ratio * width), int(ratio * height)))
    padding = ((img_size - img.shape[0], 0), (img_size - img.shape[1],0), (0,0))

    return np.pad(img, padding, 'constant')


for i in range(0,len(X_flowers)):
    
    X_flowers[i] = resize_with_pad(X_flowers[i], 360)
   # X_flowers[i] = np.array(X_flowers[i]).reshape(1, 360,360,3) useless!
    #Extra line here reshapes to accept batch size as a dimension of accepted pics at once

Not really sure what this error is, probs worth investigating, but it looks like the max height is 640. Keep in mind this data set isn't clean, there's like a python script inside it and otehr things that could explain this issue and will go away once removed. 

After making this data set the rest is using a CNN which is conveniently explained how to make below!


So basically the cells above convert this into an array, (1066, 1599, 3)  3 representing RGB vals, so that's on track with this flower recog software

Next up is to design the CNN... 

Pulling from Aymerican Dreams...




In [ ]:
plt.imshow(X_flowers[6])

In [ ]:
#should convert vals to float 32, will get on that later.
# Training Parameters
learning_rate = 0.001
num_steps = 500
batch_size = 1
display_step = 10

# Network Parameters
num_input = np.shape(X_flowers[0])[0]*np.shape(X_flowers[0])[1]  # Flower data input (img shape: x*y)
num_classes = 5 #total flower classes, daisy, etc. 5 total 
dropout = 0.5 # Dropout, probability to keep units

# tf Graph input
X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)

In [ ]:

# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')


# Create model
def conv_net(x, weights, biases, dropout):
    # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
    # Reshape to match picture format [Height x Width x Channel]
    
    ###### Tensor input become 4-D: [Batch Size, Height, Width, Channel]
    x = tf.reshape(x, shape=[-1, 360,360 , 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [ ]:
#Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, num_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

# Construct model
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)


# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [ ]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = next_batch(batch_size,X_flowers,y_flowers)
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for MNIST test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: X_flowers,
                                      Y: y_flowers}))
    

In [ ]:
%debug

In [ ]:

# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y, keep_prob: dropout})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y,
                                                                 keep_prob: 1.0})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for 256 MNIST test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: mnist.test.images[:256],
                                      Y: mnist.test.labels[:256],
                                      keep_prob: 1.0}))

In [ ]:
129600

In [ ]:
np.shape(X_flowers[0])[0]*np.shape(X_flowers[0])[1]

In [ ]:
360*360